## Divide data into train and test folders

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import shutil
import os

rootdir = '/kaggle/input/plantdisease/PlantVillage'
destroot = '/kaggle/working'
classes = [x[0].split('/')[-1] for x in os.walk(rootdir)][1:]
for i in classes:
    if not os.path.exists(destroot +'/divided/train/' + i):
        os.makedirs(destroot +'/divided/train/' + i)
    if not os.path.exists(destroot +'/divided/test/' + i):
        os.makedirs(destroot +'/divided/test/' + i)
    source = rootdir + '/' + i
    allFileNames = os.listdir(source)
    np.random.shuffle(allFileNames)
    test_ratio = 0.25
    train_FileNames, test_FileNames = np.split(np.array(allFileNames), \
    [int(len(allFileNames)* (1 - test_ratio))])
    train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]

    for name in train_FileNames:
      shutil.copy(name, destroot +'/divided/train/' + i)

    for name in test_FileNames:
      shutil.copy(name, destroot +'/divided/test/' + i)

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
train_path = '/kaggle/working/divided/train'
valid_path = '/kaggle/working/divided/test'
weight_path = '../input/keraspretrainedmodel/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

def return_string(predicted_class, classes):
    predicted = sorted(classes)[predicted_class]
    return 'This is '+ predicted + '!'
    
   


##  train InceptionResNetV2 model

In [ ]:
from tensorflow._api.v2.compat.v1 import ConfigProto
from tensorflow._api.v2.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.97
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Flatten, MaxPooling2D
from keras.models import Model, Sequential, load_model
from keras.applications.inception_resnet_v2  import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from matplotlib import pyplot as plt
# from utils import *
import os
# Import the InceptionV3  
# USE imagenet weights for Transfer Learning
IMAGE_SIZE = [299, 299]
weight_path3 = '../input/keraspretrainedmodel/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
resnet = InceptionResNetV2(input_shape=IMAGE_SIZE + [3],weights = weight_path3,input_tensor=None, include_top=False)
# we don't train existing weights. 
# We only train the last layer that will predict the predicted_classes
for layer in resnet.layers:
    layer.trainable = False
x =resnet.output
x = Flatten()(x)
prediction = Dense(15, activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)
# Specify which cost func and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Use the Image Data Generator to process images (data aug) on the fly
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#Specify the new size of the images and batch size
training_set = train_datagen.flow_from_directory(train_path,
                                                 interpolation="bicubic",
                                                 target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            interpolation="bicubic",
                                            target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                            batch_size = 32,
                                            class_mode = 'categorical')

#set up the callback to save best model's weights
checkpoint_filepath = 'InceptionResNetV2.h5'
es_callback = EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

try:
    r = model.fit_generator(
      training_set,
      validation_data=test_set,
      epochs=50,
      steps_per_epoch=len(training_set),
      validation_steps=len(test_set), 
      callbacks=[model_checkpoint_callback, es_callback]
    )
except KeyboardInterrupt:
    model.save(checkpoint_filepath)

# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('./train_val_loss_InceptionResNetV2.jpg')
plt.show()

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
#save before show
plt.savefig('./train_val_acc_InceptionResNetV2.jpg')
plt.show()
model.save(checkpoint_filepath)


## Set up a cycling Schedule for Learning Rate

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, Callback
from keras import backend as K

class CyclicLR(Callback):
    def __init__(
            self,base_lr=0.001,
            max_lr=0.006,step_size=2000.,
            mode='triangular',gamma=1.,
            scale_fn=None,scale_mode='cycle'):
        
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2','exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2.**(x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)


## Train InceptionResNetV2 Model

In [ ]:
from tensorflow._api.v2.compat.v1 import ConfigProto
from tensorflow._api.v2.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.97
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Flatten, MaxPooling2D
from keras.models import Model, Sequential, load_model
from keras.applications.inception_resnet_v2  import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from matplotlib import pyplot as plt
# from utils import *
import os
# USE imagenet weights for Transfer Learning
IMAGE_SIZE = [256, 256]
weight_path3 = '../input/keraspretrainedmodel/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
resnet = InceptionResNetV2(input_shape=IMAGE_SIZE + [3],weights = weight_path3,input_tensor=None, include_top=False)
# we don't train existing weights. 
# We only train the last layer that will predict the predicted_classes
for layer in resnet.layers:
    layer.trainable = False
x =resnet.output
x = Flatten()(x)
prediction = Dense(15, activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)
# Specify which cost func and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Use the Image Data Generator to process images (data aug) on the fly
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#Specify the new size of the images and batch size
training_set = train_datagen.flow_from_directory(train_path,
                                                 interpolation="bicubic",
                                                 target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            interpolation="bicubic",
                                            target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                            batch_size = 32,
                                            class_mode = 'categorical')

#set up the callback to save best model's weights
checkpoint_filepath = 'InceptionResNetV2_clr.h5'
es_callback = EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)

clr = CyclicLR(base_lr=0.0006, max_lr=0.001,step_size=25, mode='triangular')

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

try:
    r = model.fit_generator(
      training_set,
      validation_data=test_set,
      epochs=50,
      steps_per_epoch=len(training_set),
      validation_steps=len(test_set), 
      callbacks=[model_checkpoint_callback, clr]
    )
except KeyboardInterrupt:
    model.save(checkpoint_filepath)

# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('./train_val_loss_InceptionResNetV2_lr.jpg')
plt.show()

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
#save before show
plt.savefig('./train_val_acc_InceptionResNetV2_lr.jpg')
plt.show()
model.save(checkpoint_filepath)


## Define Resnet9 from scratch

In [ ]:
from keras.layers import ReLU,Concatenate, Dense, Conv2D, Activation, BatchNormalization, GlobalAveragePooling2D, MaxPooling2D
from keras import Sequential, activations, Input
from keras.models import Model


# convolution block with BatchNormalization
def ConvBlock(out_channels, pool=False):
    layers = [Conv2D(filters = out_channels, kernel_size=(3,3), strides=(1, 1), padding='same', kernel_regularizer=l2(1e-4), bias_regularizer=l2(1e-4)),
             BatchNormalization(axis=3,epsilon=1e-05, momentum=0.1),
             ReLU()]
    if pool:
        layers.append(MaxPooling2D(pool_size=(2, 2), strides=4, padding='valid'))
          
    return Sequential(layers)

# resnet architecture 
def ResNet9(nb_classes,IMG_SIZE):
    input_tensor = Input(shape=(IMG_SIZE[0], IMG_SIZE[1],3))   
    conv1 = ConvBlock(64)
    conv2 = ConvBlock(128, pool=True) # out_dim : 128 x 64 x 64 
    res1 = Sequential([ConvBlock(128), ConvBlock(128)])
    conv3 = ConvBlock(256, pool=True) # out_dim : 256 x 16 x 16
    conv4 = ConvBlock(512, pool=True) # out_dim : 512 x 4 x 44
    res2 = Sequential([ConvBlock(512), ConvBlock(512)])
    classifier = Sequential([MaxPooling2D(pool_size=(4, 4), strides=4, padding='valid'),
                                   Flatten(),
                                   Dense(nb_classes, activation='softmax')])
    x = input_tensor
    x = conv1(x)
    x = conv2(x)
    x = Concatenate(axis=1)([res1(x), x])
    x = conv3(x)
    x = conv4(x)
    x = Concatenate(axis=1)([res2(x) , x])
    classifier = classifier(x)
    model = Model(inputs=input_tensor, outputs=classifier)
    return model   

## Train Resnet9: This is our best Model

In [ ]:
from tensorflow._api.v2.compat.v1 import ConfigProto
from tensorflow._api.v2.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.97
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam # - Works

from keras.models import Model, Sequential, load_model
# from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from matplotlib import pyplot as plt
# from utils import *
import os
# Import the InceptionV3  
# USE imagenet weights for Transfer Learning
IMAGE_SIZE = [256, 256]

model = ResNet9(15,IMAGE_SIZE)

model.compile(
  loss='categorical_crossentropy',
  optimizer='SGD',
  metrics=['accuracy']
)

# Use the Image Data Generator to process images (data aug) on the fly
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#Specify the new size of the images and batch size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                            batch_size = 16,
                                            class_mode = 'categorical')

#set up the callback to save best model's weights
checkpoint_filepath = 'inception_resnet9.h5'
es_callback = EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)
clr = CyclicLR(base_lr=0.0006, max_lr=0.01,step_size=25, mode='triangular')

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
# fit the model
# load the model
# if os.path.exists(checkpoint_filepath):
#   model = load_model(checkpoint_filepath)
try:
    r = model.fit_generator(
      training_set,
      validation_data=test_set,
      epochs=30,
      steps_per_epoch=len(training_set),
      validation_steps=len(test_set), 
      callbacks=[model_checkpoint_callback, es_callback,clr]
    )
except:
    model.save(checkpoint_filepath)
# plot the loss
plt.plot(clr.history['loss'], label='train loss')
plt.plot(clr.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('./train_val_loss_resnet9.jpg')
plt.show()


# plot the accuracy
plt.plot(clr.history['accuracy'], label='train acc')
plt.plot(clr.history['val_accuracy'], label='val acc')
plt.legend()
#save before show
plt.savefig('./train_val_acc_resnet9.jpg')
plt.show()

model.save(checkpoint_filepath)
#See predictions: For each image, we get probabilities for every predicted_class
# y_pred = model.predict(test_set)
# #we need to choose the predicted_class with the highest prob
# y_pred = np.argmax(y_pred, axis=1)


## Training Resnet9 using Clipping values: ended up not giving better results

In [ ]:
from tensorflow._api.v2.compat.v1 import ConfigProto
from tensorflow._api.v2.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.97
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam # - Works
from keras.models import Model, Sequential, load_model
# from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

import numpy as np
from matplotlib import pyplot as plt
# from utils import *
import os
# Import the InceptionV3  
# USE imagenet weights for Transfer Learning
IMAGE_SIZE = [256, 256]
model = ResNet9(15,IMAGE_SIZE)

clipvalue=0.1
optim = Adam(clipvalue=clipvalue)
model.compile(
  loss='categorical_crossentropy',
  optimizer=optim,
  metrics=['accuracy']
)

# Use the Image Data Generator to process images (data aug) on the fly
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#Specify the new size of the images and batch size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                            batch_size = 32,
                                            class_mode = 'categorical')

#set up the callback to save best model's weights
checkpoint_filepath = 'inception_resnet9_wd.h5'
es_callback = EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)
# clr = CyclicLR(base_lr=0.0006, max_lr=0.01,step_size=15, mode='triangular')

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
# fit the model
# load the model
# if os.path.exists(checkpoint_filepath):
#   model = load_model(checkpoint_filepath)
try:
    r = model.fit_generator(
      training_set,
      validation_data=test_set,
      epochs=40,
      steps_per_epoch=len(training_set),
      validation_steps=len(test_set), 
      callbacks=[model_checkpoint_callback, es_callback,clr]
    )
except:
    model.save(checkpoint_filepath)
# plot the loss
plt.plot(clr.history['loss'], label='train loss')
plt.plot(clr.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('./train_val_loss_resnet9_wd.jpg')
plt.show()


# plot the accuracy
plt.plot(clr.history['accuracy'], label='train acc')
plt.plot(clr.history['val_accuracy'], label='val acc')
plt.legend()
#save before show
plt.savefig('./train_val_acc_resnet9_wd.jpg')
plt.show()

model.save(checkpoint_filepath)
#See predictions: For each image, we get probabilities for every predicted_class
# y_pred = model.predict(test_set)
# #we need to choose the predicted_class with the highest prob
# y_pred = np.argmax(y_pred, axis=1)


In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, Callback
from keras import backend as K

class CyclicLR(Callback):
    def __init__(
            self,base_lr=0.001,
            max_lr=0.006,step_size=2000.,
            mode='triangular',gamma=1.,
            scale_fn=None,scale_mode='cycle'):
        
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2','exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2.**(x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())
        
        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        #logs['lr'] = K.get_value(self.model.optimizer.lr)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        self.history.setdefault('epochs', []).append(epoch)


## Training InceptionV3

In [ ]:
from tensorflow._api.v2.compat.v1 import ConfigProto
from tensorflow._api.v2.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.97
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Flatten, MaxPooling2D
from keras.models import Model, Sequential, load_model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from matplotlib import pyplot as plt
# from utils import *
import os
# Import the InceptionV3  
# USE imagenet weights for Transfer Learning
IMAGE_SIZE = [256, 256]

inception = InceptionV3(input_shape=IMAGE_SIZE + [3],weights = weight_path, include_top=False)
# we don't train existing weights. 
# We only train the last layer that will predict the predicted_classes
for layer in inception.layers:
    layer.trainable = False

# Add preprocessing layer to the head of InceptionV3
#We use softmax because we have 15 predicted_classes
x = MaxPooling2D()(inception.output)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(612, activation='relu')(x)
prediction = Dense(15, activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)
# Specify which cost func and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Use the Image Data Generator to process images (data aug) on the fly
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#Specify the new size of the images and batch size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (IMAGE_SIZE[0],IMAGE_SIZE[0]),
                                            batch_size = 16,
                                            class_mode = 'categorical')

#set up the callback to save best model's weights
checkpoint_filepath = 'inception_128_More_Dense.h5'
es_callback = EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
# fit the model
# load the model
# if os.path.exists(checkpoint_filepath):
#   model = load_model(checkpoint_filepath)
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set), 
  callbacks=[model_checkpoint_callback, es_callback]
)

# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('./train_val_loss_128_More_Dense.jpg')
plt.show()


# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
#save before show
plt.savefig('./train_val_acc_128_More_Dense.jpg')
plt.show()


#load best weights
# The model weights (that are considered the best) are loaded into the model.
#model.load_weights(checkpoint_filepath)
model.save(checkpoint_filepath)
#See predictions: For each image, we get probabilities for every predicted_class
# y_pred = model.predict(test_set)
# #we need to choose the predicted_class with the highest prob
# y_pred = np.argmax(y_pred, axis=1)


## Downloading best Model

In [ ]:
from IPython.display import FileLink
FileLink(r'./resnet9.h5')

## Training InceptionV3 with different IMAGE SIZE

In [ ]:
from tensorflow._api.v2.compat.v1 import ConfigProto
from tensorflow._api.v2.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.97
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Flatten
from keras.models import Model, Sequential, load_model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from matplotlib import pyplot as plt
# from utils import *
import os
# Import the InceptionV3  
# USE imagenet weights for Transfer Learning
IMAGE_SIZE = [128, 128]

inception = InceptionV3(input_shape=IMAGE_SIZE + [3],weights = weight_path, include_top=False)
# we don't train existing weights. 
# We only train the last layer that will predict the predicted_classes
for layer in inception.layers:
    layer.trainable = False

# Add preprocessing layer to the head of InceptionV3
#We use softmax because we have 15 predicted_classes
x = Flatten()(inception.output)
prediction = Dense(15, activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)
# Specify which cost func and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Use the Image Data Generator to process images (data aug) on the fly
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#Specify the new size of the images and batch size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (128,128),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (128,128),
                                            batch_size = 64,
                                            class_mode = 'categorical')

#set up the callback to save best model's weights
checkpoint_filepath = 'inception_128.h5'
es_callback = EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
# fit the model
# load the model
# if os.path.exists(checkpoint_filepath):
#   model = load_model(checkpoint_filepath)
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set), 
  callbacks=[model_checkpoint_callback, es_callback]
)

# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('./train_val_loss_128.jpg')
plt.show()


# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
#save before show
plt.savefig('./train_val_acc_128.jpg')
plt.show()


#load best weights
# The model weights (that are considered the best) are loaded into the model.
#model.load_weights(checkpoint_filepath)
model.save(checkpoint_filepath)
#See predictions: For each image, we get probabilities for every predicted_class
# y_pred = model.predict(test_set)
# #we need to choose the predicted_class with the highest prob
# y_pred = np.argmax(y_pred, axis=1)


In [ ]:
# from tensorflow._api.v2.compat.v1 import ConfigProto
# from tensorflow._api.v2.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.97
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)
# from keras.callbacks import ModelCheckpoint
# from keras.layers import Dense, Flatten
# from keras.models import Model, Sequential, load_model
# from keras.applications.vgg import VGG16
# from keras.preprocessing.image import ImageDataGenerator
# import numpy as np
# from matplotlib import pyplot as plt
# # from utils import *
# import os
# # Import the InceptionV3  
# # USE imagenet weights for Transfer Learning
# weight_path2 = '../input/keras-efficientnetb3-no-top-weights/efficientnetb3_notop.h5'
# # inception = InceptionV3(input_shape=IMAGE_SIZE + [3],weights = weight_path, include_top=False)
# efficientNetB3 = EfficientNetB3(input_shape=IMAGE_SIZE + [3],weights = weight_path2, include_top=False)
# # we don't train existing weights. 
# # We only train the last layer that will predict the predicted_classes
# for layer in efficientNetB3.layers:
#     layer.trainable = False

# # Add preprocessing layer to the head of efficientNetB3
# #We use softmax because we have 15 predicted_classes
# x = Flatten()(efficientNetB3.output)
# prediction = Dense(15, activation='softmax')(x)

# # create a model object
# model = Model(inputs=efficientNetB3.input, outputs=prediction)
# # Specify which cost func and optimization method to use
# model.compile(
#   loss='categorical_crossentropy',
#   optimizer='adam',
#   metrics=['accuracy']
# )

# # Use the Image Data Generator to process images (data aug) on the fly
# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

# test_datagen = ImageDataGenerator(rescale = 1./255)

# #Specify the new size of the images and batch size
# training_set = train_datagen.flow_from_directory(train_path,
#                                                  target_size = (224, 224),
#                                                  batch_size = 32,
#                                                  class_mode = 'categorical')
# test_set = test_datagen.flow_from_directory(valid_path,
#                                             target_size = (224, 224),
#                                             batch_size = 32,
#                                             class_mode = 'categorical')

# #set up the callback to save best model's weights
# checkpoint_filepath = 'EfficientNetB3.h5'
# model_checkpoint_callback = ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=False,
#     monitor='val_acc',
#     mode='max',
#     save_best_only=True)
# # fit the model
# # load the model
# # if os.path.exists(checkpoint_filepath):
# #   model = load_model(checkpoint_filepath)
# r = model.fit_generator(
#   training_set,
#   validation_data=test_set,
#   epochs=50,
#   steps_per_epoch=len(training_set),
#   validation_steps=len(test_set), 
#   callbacks=[model_checkpoint_callback]
# )

# # plot the loss
# plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
# plt.legend()
# plt.savefig('./train_val_loss2.jpg')
# plt.show()


# # plot the accuracy
# plt.plot(r.history['accuracy'], label='train acc')
# plt.plot(r.history['val_accuracy'], label='val acc')
# plt.legend()
# #save before show
# plt.savefig('./train_val_acc2.jpg')
# plt.show()


# #load best weights
# # The model weights (that are considered the best) are loaded into the model.
# #model.load_weights(checkpoint_filepath)
# model.save(checkpoint_filepath)
# #See predictions: For each image, we get probabilities for every predicted_class
# # y_pred = model.predict(test_set)
# # #we need to choose the predicted_class with the highest prob
# # y_pred = np.argmax(y_pred, axis=1)
